In [7]:
from jupyter_plotly_dash import JupyterDash

import base64
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from CRUD import AnimalShelter





###########################
# Data Manipulation / Model
###########################

username = "daniyo"
password = "123456"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')


image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())




app.layout = html.Div([
    html.A([
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
    height = 100, width = 100))]),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Dani Yosopov CS-340 Dashboard'))),
    html.Hr(),
        
 dcc.RadioItems(
        id='filter-type',
        options=[
            {'label':'Water Rescue', 'value': 'Water'},
            {'label':'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label':'Disaster Rescue or Individual Tracking', 'value':'Disaster'},
            {'label': 'Reset Table', 'value': 'Reset'},
        ],
        value='Reset'
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action='native',
        sort_action='native',
        sort_mode='multi',
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_rows=[],
        page_action = "native",
        page_current = 0,
        page_size = 10
        
    ),
    html.Br(),
    html.Hr(),
        
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
    children=[
        html.Div(
            #dcc.Graph(id='graph-id'),
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
#############################################
# Interaction Between Components / Controller
#############################################



@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
    if filter_type == 'Reset':
        df = pd.DataFrame.from_records(shelter.read({}))
        
    elif filter_type == 'Water':
        df = pd.DataFrame.from_records(shelter.read({'$or':[
                {'breed': 'Labrador Retriever'},
                {'breed': 'Newfoundland'}
        ]}))
        
    elif filter_type == 'Mountain':
        df = pd.DataFrame.from_records(shelter.read({'$or':[
                {'breed': 'Bernese Mountain'},
                {'breed': 'Great Pyrenees'},
                {'breed': 'Greater Swiss Mountain'},
                {'breed': 'Kuvasz'},
                {'breed': 'Newfoundland'}
        ]}))
    
    elif filter_type == 'Disaster':
        df = pd.DataFrame.from_records(shelter.read({'$or':[
                {'breed': 'Belgian Malinois'},
                {'breed': 'Basset Hound'},
                {'breed': 'St. Bernard'},
                {'breed': 'Beagle'},
                {'breed': 'Labrador Retriever'}
        ]}))
    
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    graph = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(            
            figure = px.scatter(graph, x='breed', y="outcome_type")
        )    
    ]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]




app